# Easily log function calls

In [2]:
import logging
from functools import wraps
logging.basicConfig(level=logging.DEBUG)
 
def logging_wrapper(func):
    @wraps(func)
    def logging_inner(*args, **kwargs):
        kwargs_formatted = ['{}={}'.format(k, repr(v)) for k, v in kwargs.items()]
        call_line = 'Func call: {}({})'.format(func.__name__, 
                                    ', '.join([repr(v) for v in args] + kwargs_formatted))
        try:
            res = func(*args, **kwargs)
            logging.debug('{} - returns {}'.format(call_line, repr(res)))
            return res
        except Exception as exc:
            logging.exception(call_line + ' caused exception!')
    return logging_inner

#### Now, decorate function of your choice


In [3]:
@logging_wrapper
def add(x, y):
    return x + y

#### Shall we test?  


In [4]:
add(1, 4)

DEBUG:root:Func call: add(1, 4) - returns 5


5

In [5]:
add(1, '4')

ERROR:root:Func call: add(1, '4') caused exception!
Traceback (most recent call last):
  File "<ipython-input-2-492f65d61994>", line 12, in logging_inner
    res = func(*args, **kwargs)
  File "<ipython-input-3-5e4c59fc6abd>", line 3, in add
    return x + y
TypeError: unsupported operand type(s) for +: 'int' and 'str'


#### But what about named arguments?


In [6]:
@logging_wrapper
def add(first, second=None):
    return first + second

In [7]:
add(1,4)

DEBUG:root:Func call: add(1, 4) - returns 5


5

In [8]:
add(1, second=4)


DEBUG:root:Func call: add(1, second=4) - returns 5


5

In [9]:
add(1, second='4')

ERROR:root:Func call: add(1, second='4') caused exception!
Traceback (most recent call last):
  File "<ipython-input-2-492f65d61994>", line 12, in logging_inner
    res = func(*args, **kwargs)
  File "<ipython-input-6-7bd47bc7e5b7>", line 3, in add
    return first + second
TypeError: unsupported operand type(s) for +: 'int' and 'str'


# Static variables

#### No such animal in Python?! 

#### Remember, you were told never to use default mutable values - but the language allows them?

In [10]:
def count_calls(_counter=[0]):
    _counter[0] += 1
    print (f'I was called {_counter[0]} times')

In [13]:
count_calls()

I was called 3 times


#### What about function attribute?

In [14]:
def count_calls_take2():
    count_calls_take2.cnt += 1
    print (f'I was called {count_calls_take2.cnt} times')
setattr(count_calls_take2, 'cnt', 0)

In [17]:
count_calls_take2()

I was called 3 times


It works, but it looks ugly. And if you need more than 1 "static" variable?

#### But what if you could do it in more elegant way?

In [18]:
def static_variables(**static_kw):
    def static_setter(func):
        for static_name, init_value in static_kw.items():
            setattr(func, static_name, init_value)
        return func
    return static_setter

#### Now, just put it to work

In [19]:
@static_variables(cnt=0)
def count_calls_take3():
    count_calls_take3.cnt +=1
    print(f'I was called {count_calls_take3.cnt} times')

In [22]:
count_calls_take3()

I was called 3 times


# Plurals for grammar freaks

There's a great standard decorator _singledispatch_ that allows your to create dispatcher on function's first parameter. Sort of like function overloading.

Let us start by creating actual hard-working function that will try to define the proper plural form.

In [24]:
import collections
from functools import singledispatch

def _make_plural(count, word):
    if count == 1:
        pass
    elif word[-1] in 'sxz' or word[-2:] in {'sh', 'ch'}:
        word += 'es'
    elif word[-1] == 'y' and word[-2] not in 'aoeui':
        word = word[:-1] + 'ies'
    elif word[-1] == 'f':
        word = word[:-1] + 'ves' 
    elif word[-2:] == 'fe':
        word = word[:-2] + 'ves'
    else:
        word += 's'
    return f'{count} {word}'

#### Next step - we want to be able to use it both on numbers and iterables - lists, e.g.

In [25]:
@singledispatch
def plural(value, value_descr):
    return _make_plural(value, value_descr)

@plural.register(collections.Iterable)
def _(value, value_descr):
    return _make_plural(len(value), value_descr)

#### Will it work?

In [26]:
print(plural(1, 'orange'),
      plural(2, 'apple'),
      plural(3, 'baby'),
      plural(4, 'match'),
      plural(5, 'leaf'),
      plural(6, 'knife'),
      plural(['apple', 'orange', 'grapefruit'], 'fruit'),
      sep='\n')

1 orange
2 apples
3 babies
4 matches
5 leaves
6 knives
3 fruits


# Self organizing class

#### What kind of animal is that?

* You have an instance **self_organized** of a class **SelfOrganized** with methods **a**, **b**, **c** and **d**

* You need to call **a**,**b**, **c** and **d** in that order.

* You want to add new methods **e** and **f** to the class - and add them to the execution list

* You want to make adding new methods to the class - and to the execution list - as hassle-free as possible

## The reason?

* From my career - adding methods displaying widgets on image

  * Widget list was growing

  * I had no desire to distribute widgets "manually" 

* Interview project with textual menu
    * I wanted to show off
    * ... and I did not get the job (for another reason)

## How?

 * Create a decorator for self-arranging methods ([SO credit](https://stackoverflow.com/a/12718272/1381627))
```python   
@staticmethod
def _make_menu_item(func, item_no=[0]):
    """ Method decorator
        Mark decorated function as menu item by setting attribute
        _item_idx and use it to set order
    """
    item_no[0] += 1
    setattr(func, '_menu_idx', item_no[0])
    return func
_menu_enumerator = _make_menu_item.__func__```

* Define methods to be part of self-arrangement

```python
@_menu_enumerator
def one(self, *args, **kwargs):
    print('You chose option 1. One is lonely.')

@_menu_enumerator
def two(self, *args, **kwargs):
    print('You chose option 2. Two\'s company.')
```

* Collect self-organizining methods
```python
def __init__(self):
    all_attribs = (getattr(self, attr) for attr in dir(self))
    menu_items = sorted((attr for attr in all_attribs 
                         if hasattr(attr, '_menu_idx')),
                        key=lambda a: a._menu_idx)
    self._menu = OrderedDict((str(m._menu_idx), m) 
                             for m in menu_items)```

* Wrap function calls
```python
    def __call__(self):
        while True:
            print(self._menu_prompts)
            choice = input('Your choice? ')
            end_game = self._menu.get(choice, self.bad_choice)()
            if end_game:
                break
```

### And now - the grand finale!

In [29]:
from collections import OrderedDict

class AutoMenu:
    def __init__(self):
        """
        Menu is built by collecting functions decorated with "_menu_enumerator"
        """
        all_attribs = (getattr(self, attr) for attr in dir(self))
        menu_items = sorted((attr for attr in all_attribs if hasattr(attr, '_menu_idx')),
                            key=lambda a: a._menu_idx)
        self._menu_prompts = '\nMake your choice\n' + '\n'.join(
            '{:2d} - {}'.format(f._menu_idx, f.__name__.replace('_', ' ').title())
            for f in menu_items)
        self._menu = OrderedDict((str(m._menu_idx), m) for m in menu_items)

    @staticmethod
    def _make_menu_item(func, item_no=[0]):
        """ Method decorator
            Mark decorated function as menu item by setting attribute
            _item_idx and use it to set order
        """
        item_no[0] += 1
        setattr(func, '_menu_idx', item_no[0])
        return func

    _menu_enumerator = _make_menu_item.__func__

    @_menu_enumerator
    def one(self, *args, **kwargs):
        print('You chose option 1. One is lonely.')

    @_menu_enumerator
    def two(self, *args, **kwargs):
        print('You chose option 2. Two\'s company.')

    @_menu_enumerator
    def four(self, *args, **kwargs):
        print('You chose option 4. Four is too much.')

    @_menu_enumerator
    def three(self, *args, **kwargs):
        print('You chose option 3. Three\'s a crowd.')

    @_menu_enumerator
    def end_game(self, *args, **kwargs):
        print('You chose to exit. Farewell.')
        return True

    def bad_choice(self):
        print('Are you sure?')

    def __call__(self):
        while True:
            print(self._menu_prompts)
            choice = input('Your choice? ')
            end_game = self._menu.get(choice, self.bad_choice)()
            if end_game:
                break

For more fun stuff, see Facebook [Python Programmin Language](https://www.facebook.com/groups/python.programmers/) group.